Cloning the MuyGPs library

In [ ]:
!pip install git+https://github.com/LLNL/MuyGPyS@develop

Importing some of the MuyGPs commands

In [ ]:
import numpy as np

from MuyGPyS._test.sampler import UnivariateSampler2D, print_results
from MuyGPyS.gp import MuyGPS
from MuyGPyS.gp.deformation import Isotropy, l2
from MuyGPyS.gp.hyperparameter import AnalyticScale, Parameter
from MuyGPyS.gp.kernels import Matern
from MuyGPyS.gp.noise import HomoscedasticNoise
from MuyGPyS.gp.tensors import make_predict_tensors, make_train_tensors
from MuyGPyS.neighbors import NN_Wrapper
from MuyGPyS.optimize import Bayes_optimize
from MuyGPyS.optimize.batch import sample_batch
from MuyGPyS.optimize.loss import lool_fn, looph_fn

In [ ]:
#Running the functions to be applied
%run test_looph.py
%run test_looph_mean.py
%run lool_hybrid.py
%run looph_hybrid.py
%run DownS_lool.py
%run DownS_looph.py

Sampling a 2D surface from a conventional Gaussian Process

In [ ]:
sampler = UnivariateSampler2D(
    points_per_dim=100,
    train_ratio=0.90,
    kernel=Matern(
        smoothness=Parameter(1.0),
        deformation=Isotropy(
            l2,
            length_scale=Parameter(1.0)
        ),
    ),
    noise=HomoscedasticNoise(1e-14),
    measurement_noise=HomoscedasticNoise(1e-7),
)

Train/test data split

In [ ]:
train_features, test_features= sampler.features()
train_responses, test_responses = sampler.sample()

Injecting outliers in the train target using 10% portion of the observations

In [ ]:
#np.random.seed(55)
idx = np.random.choice(np.arange(train_features.shape[0]), size=int(0.10 * train_features.shape[0]), replace=False)

multi = np.array([2, 3, 5])
yout_train = train_responses.copy()
yout_train[idx] = train_responses[idx] * multi[2]